Lambda School Data Science

*Unit 2, Sprint 3, Module 2*

---


# Wrangle ML datasets 
- Explore tabular data for supervised machine learning
- Join relational data for supervised machine learning

# Explore tabular data for supervised machine learning 🍌

Wrangling your dataset is often the most challenging and time-consuming part of the modeling process.

In today's lesson, we’ll work with a dataset of [3 Million Instacart Orders, Open Sourced](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2)!

Let’s get set up:

In [1]:
# Download data
import requests

def download(url):
    filename = url.split('/')[-1]
    print(f'Downloading {url}')
    r = requests.get(url)
    with open(filename, 'wb') as f:
        f.write(r.content)
    print(f'Downloaded {filename}')

download('https://lambdaschool-ds-instruction.s3.amazonaws.com/datasets%3Ainstacart_2017_05_01.tar.gz')

Downloaded datasets%3Ainstacart_2017_05_01.tar.gz


In [2]:
# Uncompress data
import tarfile
tarfile.open('datasets%3Ainstacart_2017_05_01.tar.gz').extractall()

In [3]:
# Change directory to where the data was uncompressed
%cd instacart_2017_05_01

/Users/petr/Documents/Lambda/Units/Unit 2/Sprint 3/DS-Unit-2-Applied-Modeling/module2-wrangle-ml-datasets/instacart_2017_05_01


In [4]:
# Print the csv filenames
from glob import glob
for filename in glob('*.csv'):
    print(filename)

products.csv
orders.csv
order_products__train.csv
departments.csv
aisles.csv
order_products__prior.csv


In [7]:
# For each csv file, look at its shape & head
import pandas as pd
from IPython.display import display


def preview():
    for filename in glob('*.csv'):
        df = pd.read_csv(filename)
        print(filename, df.shape)
        display(df.head())
preview()

products.csv (49688, 4)


,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


orders.csv (3421083, 7)


,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


order_products__train.csv (1384617, 4)


...

,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol


aisles.csv (134, 2)


,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars
3,4,instant foods
4,5,marinades meat preparation


order_products__prior.csv (32434489, 4)


,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


### The original task was complex ...

[The Kaggle competition said,](https://www.kaggle.com/c/instacart-market-basket-analysis/data):

> The dataset for this competition is a relational set of files describing customers' orders over time. The goal of the competition is to predict which products will be in a user's next order.

> orders.csv: This file tells to which set (prior, train, test) an order belongs. You are predicting reordered items only for the test set orders.

Each row in the submission is an order_id from the test set, followed by product_id(s) predicted to be reordered.

> sample_submission.csv: 
```
order_id,products
17,39276 29259
34,39276 29259
137,39276 29259
182,39276 29259
257,39276 29259
```

### ... but we can simplify!

Simplify the question, from "Which products will be reordered?" (Multi-class, [multi-label](https://en.wikipedia.org/wiki/Multi-label_classification) classification) to **"Will customers reorder this one product?"** (Binary classification)

Which product? How about **the most frequently ordered product?**

### Questions:

- What is the most frequently ordered product?
- How often is this product included in a customer's next order?
- Which customers have ordered this product before?
- How can we get a subset of data, just for these customers?
- What features can we engineer? We want to predict, will these customers reorder this product on their next order?

## Follow Along

### What was the most frequently ordered product?

In [14]:
order_products__train=pd.read_csv('order_products__train.csv')
order_products__train['product_id'].value_counts()

24852    18726
13176    15480
21137    10894
21903     9784
47626     8135
         ...  
22373        1
25364        1
26467        1
41652        1
27193        1
Name: product_id, Length: 39123, dtype: int64

In [20]:
temp = order_products__train.sort_values('product_id', ascending=False)
temp

,order_id,product_id,add_to_cart_order,reordered
891142,2198380,49688,10,0
1209047,2986153,49688,16,0
269349,655800,49688,10,1
1014644,2508423,49688,3,0
446055,1092104,49687,1,0
...,...,...,...,...
507205,1245784,1,7,0
97272,240996,1,3,1
771078,1897200,1,24,1
19960,48361,1,9,0


In [23]:
# Let us group by product_id and get a count of how many times a product was ordered and count them
temp.groupby('product_id').count()

,order_id,add_to_cart_order,reordered
product_id,,,
24852,18726,18726,18726
13176,15480,15480,15480
21137,10894,10894,10894
21903,9784,9784,9784
47626,8135,8135,8135
...,...,...,...
33470,1,1,1
19973,1,1,1
19969,1,1,1


In [24]:
products_ordered = order_products__train.groupby('product_id').count().sort_values('order_id', ascending=False)

,order_id,add_to_cart_order,reordered
product_id,,,
24852,18726,18726,18726
13176,15480,15480,15480
21137,10894,10894,10894
21903,9784,9784,9784
47626,8135,8135,8135
...,...,...,...
33470,1,1,1
19973,1,1,1
19969,1,1,1


In [28]:
products = pd.read_csv('products.csv')
products[products.product_id == 24852]

,product_id,product_name,aisle_id,department_id
24851,24852,Banana,24,4


In [33]:
# Creating our train dataset by erging orders with the products

train = pd.merge(order_products__train, products, on='product_id', how='inner')
train.head()

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id
0,1,49302,1,1,Bulgarian Yogurt,120,16
1,816049,49302,7,1,Bulgarian Yogurt,120,16
2,1242203,49302,1,1,Bulgarian Yogurt,120,16
3,1383349,49302,11,1,Bulgarian Yogurt,120,16
4,1787378,49302,8,0,Bulgarian Yogurt,120,16


### How often are bananas included in a customer's next order?

There are [three sets of data](https://gist.github.com/jeremystan/c3b39d947d9b88b3ccff3147dbcf6c6b):

> "prior": orders prior to that users most recent order (3.2m orders)  
"train": training data supplied to participants (131k orders)  
"test": test data reserved for machine learning competitions (75k orders)

Customers' next orders are in the "train" and "test" sets. (The "prior" set has the orders prior to the most recent orders.)

We can't use the "test" set here, because we don't have its labels (only Kaggle & Instacart have them), so we don't know what products were bought in the "test" set orders.

So, we'll use the "train" set. It currently has one row per product_id and multiple rows per order_id.

But we don't want that. Instead we want one row per order_id, with a binary column: "Did the order include bananas?"

Let's wrangle!

In [34]:
# let's just look at a single order to get a handle of what we need to do
train[train['order_id'] == 816049]

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id
1,816049,49302,7,1,Bulgarian Yogurt,120,16
26911,816049,22035,14,0,Organic Whole String Cheese,21,16
47740,816049,20574,2,1,Roasted Turkey,96,20
196442,816049,41220,4,0,Organic Romaine Lettuce,83,4
264132,816049,34969,1,1,Red Vine Tomato,83,4
355646,816049,7503,15,0,Whole Almonds,117,19
363747,816049,19446,16,0,Thick & Chunky Salsa,67,20
370530,816049,44142,8,0,Red Onion,83,4
552816,816049,36772,12,0,Bunny Pasta with Yummy Cheese Macaroni & Cheese,4,9
603546,816049,26604,11,0,Organic Blackberries,24,4


In [36]:
# Let's come up with a boolean for whether an order has a banana or not

train['has_banana'] = train['product_name'] =='Banana'
train.head()

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,has_banana
0,1,49302,1,1,Bulgarian Yogurt,120,16,False
1,816049,49302,7,1,Bulgarian Yogurt,120,16,False
2,1242203,49302,1,1,Bulgarian Yogurt,120,16,False
3,1383349,49302,11,1,Bulgarian Yogurt,120,16,False
4,1787378,49302,8,0,Bulgarian Yogurt,120,16,False


In [37]:
# How many of the products were bananas?

train['has_banana'].value_counts(normalize=True)

False    0.986476
True     0.013524
Name: has_banana, dtype: float64

In [48]:
train.shape

(1384617, 8)

In [47]:
train_wrangled = train.groupby('order_id').any()
train_wrangled

,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,has_banana
order_id,,,,,,,
1,True,True,True,True,True,True,False
36,True,True,True,True,True,True,False
38,True,True,True,True,True,True,False
96,True,True,True,True,True,True,False
98,True,True,True,True,True,True,False
...,...,...,...,...,...,...,...
3421049,True,True,True,True,True,True,False
3421056,True,True,True,True,True,True,False
3421058,True,True,True,True,True,True,False


In [49]:
# how many of the ordered contained bananas

train_wrangled = train.groupby('order_id')['has_banana'].any().reset_index()
train_wrangled['has_banana'].value_counts(normalize=True)

# 14% of the orderes had bananas

False    0.857281
True     0.142719
Name: has_banana, dtype: float64

# Join relational data for supervised machine learning

## Overview
Often, you’ll need to join data from multiple relational tables before you’re ready to fit your models.

### Which customers have ordered this product before?

- Customers are identified by `user_id`
- Products are identified by `product_id`

Do we have a table with both these id's? (If not, how can we combine this information?)

In [ ]:
# We do not yet have a table that contains the user_id and product_id
# orders.csv has the user_id and order_id
# order_products contains order_id and product_id

previous_orders = pd.merge(order_products__train, products, on='product_id', how='inner')

In [51]:
banana_order_ids = train[train['has_banana']==True]['order_id']
banana_order_ids

129688        226
129689        473
129690        878
129691       1042
129692       1139
           ...   
148409    3419531
148410    3419542
148411    3419629
148412    3420088
148413    3420996
Name: order_id, Length: 18726, dtype: int64

In [63]:
# Let's look at the order table

orders = pd.read_csv('orders.csv')

# Getting the subset of the orders table which contain bananas

banana_orders = orders[orders['order_id'].isin(banana_order_ids)]
orders.shape, banana_orders.shape

((3421083, 7), (18726, 7))

In [70]:
# Now get the user_ids of everyone who has bought bananas!

banana_user_ids = banana_orders['user_id']
banana_user_ids

25              2
485            34
623            43
652            46
881            56
            ...  
3420581    206181
3420692    206191
3420786    206196
3420933    206205
3421082    206209
Name: user_id, Length: 18726, dtype: int64

## Follow Along

### How can we get a subset of data, just for these customers?

We want *all* the orders from customers who have *ever* bought bananas.

(And *none* of the orders from customers who have *never* bought bananas.)

In [75]:
orders_customers_who_bought_bananas = orders[orders['user_id'].isin(banana_user_ids)]
orders_customers_who_bought_bananas.shape

(332661, 7)

### What features can we engineer? We want to predict, will these customers reorder bananas on their next order?

In [ ]:
# products per order
# time of the dat of the order (breaking apart the datetime)
# have they reordered bananas before?
# size of the order
# frequency of banana orders

## Challenge

**Continue to clean and explore your data.** Can you **engineer features** to help predict your target? For the evaluation metric you chose, what score would you get just by guessing? Can you **make a fast, first model** that beats guessing?

We recommend that you use your portfolio project dataset for all assignments this sprint. But if you aren't ready yet, or you want more practice, then use the New York City property sales dataset today. Follow the instructions in the assignment notebook. [Here's a video walkthrough](https://youtu.be/pPWFw8UtBVg?t=584) you can refer to if you get stuck or want hints!